# Importing Libraries

In [ ]:
import numpy as np
from tensorflow.keras import regularizers
import math
import tensorflow as tf
import tensorflow.keras.layers as tfl
from keras import backend as K 

# Loading Data

In [ ]:
from tensorflow.keras.datasets import cifar100

(X_train, y_train), (X_test, y_test) = cifar100.load_data(label_mode='coarse')

In [ ]:
print(X_train.shape,y_train.shape)

# Resizing images

In [ ]:
import cv2
X_train = np.array([cv2.resize(img, (140, 140)) for img in X_train]) 

In [ ]:
X_test = np.array([cv2.resize(img, (140, 140)) for img in X_test])

# Encoding labels

In [ ]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()
y_train=enc.fit_transform(y_train).toarray().astype(int)
y_test=enc.transform(y_test).toarray().astype(int)


print(y_train.shape)
print(y_train[0])

# Data Augmentation

In [ ]:
def data_augmenter():
    '''
    Create a Sequential model composed of 2 layers
    Returns:
        tf.keras.Sequential
    '''
    data_augmentation = tf.keras.Sequential()
    data_augmentation.add(tfl.RandomFlip('horizontal'))
    data_augmentation.add(tfl.RandomRotation(0.2))

    return data_augmentation

In [ ]:
data_augmentation = data_augmenter()

# AlexNet architecture 

In [ ]:
def arch(input_shape, data_augmentation):

    input_img = tf.keras.Input(shape=input_shape)
    layer = data_augmenter()(input_img)
    layer=tfl.Conv2D(filters= 96 , kernel_size= 11,strides=(4, 4))(layer)
    layer=tfl.ReLU()(layer)
    layer=tfl.MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding='same')(layer)
    layer=tfl.BatchNormalization()(layer,training=True)
    
    layer=tfl.Conv2D(filters= 256 , kernel_size= 5 ,strides=(1, 1), padding='same')(layer)
    layer=tfl.ReLU()(layer)
    layer=tfl.MaxPool2D(pool_size=(3, 3), strides=(2,2), padding='same')(layer)
    layer=tfl.BatchNormalization()(layer,training=True)

    layer=tfl.Conv2D(filters= 384 , kernel_size= 3 ,strides=(1, 1), padding='same')(layer)
    layer=tfl.ReLU()(layer)
    layer=tfl.Conv2D(filters= 384 , kernel_size= 3 ,strides=(1, 1), padding='same')(layer)
    layer=tfl.ReLU()(layer)
    layer=tfl.Conv2D(filters= 256 , kernel_size= 3 ,strides=(1, 1), padding='same')(layer)
    layer=tfl.ReLU()(layer)
    layer=tfl.MaxPool2D(pool_size=(3, 3), strides=(2,2), padding='same')(layer)
    layer=tfl.BatchNormalization()(layer,training=True)

    layer=tfl.Flatten()(layer)
    
    layer=tfl.Dense(units=4096, activation='relu')(layer)
    layer=tfl.Dropout(0.2)(layer)
    layer=tfl.BatchNormalization()(layer,training=True)

    layer=tfl.Dense(units=4096, activation='relu')(layer)
    layer=tfl.Dropout(0.2)(layer)
    layer=tfl.BatchNormalization()(layer,training=True)

    layer=tfl.Dense(units=1000, activation='relu')(layer)
    layer=tfl.Dropout(0.2)(layer)
    layer=tfl.BatchNormalization()(layer,training=True)

    outputs=tfl.Dense(units= 20 , activation='softmax')(layer)
    model = tf.keras.Model(inputs=input_img, outputs=outputs)
    return model

# training and evaluating the model

In [ ]:
conv_model = arch((140, 140, 3),data_augmentation)
conv_model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
conv_model.summary()

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(64)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(64)
history = conv_model.fit(train_dataset,epochs=10,validation_data=test_dataset,batch_size=64,shuffle=True)